# SCN with Multi Product

Pada sesi ini kita akan menyelesaikan salah satu kasus rantai pasok di mana terdapat dua buah produk yang harus dikirimkan. Data yang saya gunakan bersumber dari praktikum rantai pasok yang saya dapatkan di semester 5. Saya akan mencoba menyelesaikannya menggunakan PuLP.

**Permasalahan:**

Sebuah perusahaan memproduksi dua jenis produk yakni kursi (A) dan meja (B) di tiga pabrik. Produk B memiliki ukuran **dua kali lebih besar** daripada produk A. Produk dapat dikirim langsung dari pabrik menuju retailer atau melalui gudang (gudang bersifat cross docking). Berikut adalah gambar konfigurasi jaringan rantai pasok perusahaan tersebut:


<img src="https://user-images.githubusercontent.com/61647791/147574241-87712480-b24f-437d-8b48-66bd3f8eae4c.png" />


Biaya yang digunakan untuk mengirimkan kedua produk di setiap pabrik diasumsikan sama sehingga akan saling berkompetisi dalam memanfaatkan kapasitas transportasi yang tersedia. Setiap pabrik memiliki kapasitas produksi yang berbeda, begitupun dengan permintaan di masing-masing retailer.

## 1. Mengimpor Library dan Data

In [1]:
import numpy as np
import pandas as pd
from pulp import *

#### Costs
Tabel ini berisikan biaya transportasi produk A maupun B per unit dari lokasi i ke j.

In [2]:
cost = pd.read_excel("SCN with Multi Product.xlsx",sheet_name="Cost").set_index("From/To")
cost

,Plant 1,Plant 2,Plant 3,Warehouse 1,Warehouse 2,Retailer 1,Retailer 2
From/To,,,,,,,
Plant 1,NaN,5.0,3.0,5.0,5.0,20.0,20.0
Plant 2,9.0,NaN,9.0,1.0,1.0,8.0,15.0
Plant 3,0.4,8.0,NaN,1.0,0.5,10.0,12.0
Warehouse 1,NaN,NaN,NaN,NaN,1.2,2.0,12.0
Warehouse 2,NaN,NaN,NaN,0.8,NaN,2.0,12.0
Retailer 1,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Retailer 2,NaN,NaN,NaN,NaN,NaN,7.0,NaN


#### Capacity
Tabel ini berisikan kapasitas produksi pada produk A dan B di setiap pabrik.

In [3]:
capacity = pd.read_excel("SCN with Multi Product.xlsx",sheet_name="Plant Capacity").set_index("Plant")
capacity

,Product A,Product B
Plant,,
Plant 1,100,100
Plant 2,150,150
Plant 3,50,50


#### Demand
Tabel ini berisikan permintaan produk A maupun B pada masing-masing retailer.

In [4]:
demand = pd.read_excel("SCN with Multi Product.xlsx",sheet_name="Demand").set_index("Retailer")
demand

,Product A,Product B
Retailer,,
Retailer 1,200,200
Retailer 2,90,90


#### Weight
Dictionary ini menunjukkan perbandingan ukuran produk A dan B.

In [5]:
weight = {"Product A":1,
          "Product B":2}

## 2. Inisiasi Model
Tujuan dari permasalahan ini adalah meminimalkan total biaya transportasi sehingga kita akan menggunakan LpMinimize.

In [6]:
model = LpProblem("SCN_with_multiproduct", LpMinimize)

## 3. Menambahkan Variabel


Variabel keputusan yang ada di model ini yaitu Xcij yang menunjukkan jumlah produk A atau B yang dikirimkan dari lokasi i ke j. Variabel ini memiliki tipe integer lebih dari sama dengan nol dan kurang dari sama dengan kapasitas transportasi.

<img src="https://user-images.githubusercontent.com/61647791/147671446-e30cd497-d9de-4656-8b4a-3de447c514e5.png" />



### <font color='red'>Hati-hati!</font>
Kita harus melakukan filter pada variable Xcij terlebih dahulu untuk kemudahan akses pada fungsi tujuan dan constrain dengan beberapa ketentuan sebagai berikut:
1. Tidak ada pengiriman dari i ke j di mana i = j
2. Tidak ada pengiriman produk dari gudang ke pabrik
3. Tidak ada pengiriman produk dari retailer ke pabrik
4. Tidak ada pengiriman produk dari retailer ke gudang

Pada tabel biaya ditunjukkan dengan adanya null value atau NaN.

In [7]:
plant = [p for p in cost.index if p[:5]=="Plant"]
warehouse = [w for w in cost.index if w[:9]=="Warehouse"]
retailer = [r for r in cost.index if r[:8]=="Retailer"]

wh_to_p = [(i,j) for i in warehouse for j in plant]
r_to_p = [(i,j) for i in retailer for j in plant]
r_to_wh = [(i,j) for i in retailer for j in warehouse]

nan_keys = wh_to_p + r_to_p + r_to_wh

product = ["Product A", "Product B"]

keys = [(c,i,j) for c in product for i in cost.index for j in cost.index if i!=j and (i,j) not in nan_keys]
keys

[('Product A', 'Plant 1', 'Plant 2'),
 ('Product A', 'Plant 1', 'Plant 3'),
 ('Product A', 'Plant 1', 'Warehouse 1'),
 ('Product A', 'Plant 1', 'Warehouse 2'),
 ('Product A', 'Plant 1', 'Retailer 1'),
 ('Product A', 'Plant 1', 'Retailer 2'),
 ('Product A', 'Plant 2', 'Plant 1'),
 ('Product A', 'Plant 2', 'Plant 3'),
 ('Product A', 'Plant 2', 'Warehouse 1'),
 ('Product A', 'Plant 2', 'Warehouse 2'),
 ('Product A', 'Plant 2', 'Retailer 1'),
 ('Product A', 'Plant 2', 'Retailer 2'),
 ('Product A', 'Plant 3', 'Plant 1'),
 ('Product A', 'Plant 3', 'Plant 2'),
 ('Product A', 'Plant 3', 'Warehouse 1'),
 ('Product A', 'Plant 3', 'Warehouse 2'),
 ('Product A', 'Plant 3', 'Retailer 1'),
 ('Product A', 'Plant 3', 'Retailer 2'),
 ('Product A', 'Warehouse 1', 'Warehouse 2'),
 ('Product A', 'Warehouse 1', 'Retailer 1'),
 ('Product A', 'Warehouse 1', 'Retailer 2'),
 ('Product A', 'Warehouse 2', 'Warehouse 1'),
 ('Product A', 'Warehouse 2', 'Retailer 1'),
 ('Product A', 'Warehouse 2', 'Retailer 2'),
 (

In [8]:
#Decision Variable
#Truk hanya dapat mengangkut sebanyak 300 unit
x = LpVariable.dicts("x", keys, lowBound=0, upBound=300, cat="Integer")

## 4. Menambahkan Fungsi Tujuan dan Constrain
Berikut adalah fungsi tujuan dari permasalahan ini: 


<img src="https://user-images.githubusercontent.com/61647791/147672112-61706693-90bb-4c80-8e73-1dfdc4f5184e.png" />

In [9]:
#Objective Function
model += lpSum(weight[c]*x[c,i,j]*cost.loc[i,j] for (c,i,j) in keys)

#### Constrain 1: Production Capacity Constrain
Selisih masing-masing produk yang keluar dan masuk sebuah pabrik harus kurang dari atau sama dengan kapasitasnya. Produk yang masuk ke pabrik bisa berasal dari pabrik lainnya. Produk dari pabrik dapat dikirimkan ke pabrik lain, gudang maupun langsung menuju retailer. 

<img src="https://user-images.githubusercontent.com/61647791/147672623-1a84c392-8d44-4935-b555-e2c8f2ebcbb2.png" />

In [10]:
#Production Capacity Constrain
for c in product:
    for p in plant:
        model += lpSum(x[c,p,j] for j in plant+warehouse+retailer if p!=j) - lpSum(x[c,i,p] for i in plant if i!=p) <= capacity.loc[p][c]

#### Constrain 2: Warehouse Cross-Docking
Dalam kasus ini gudang bersifat cross docking yang berarti hanya berupa penyimpanan sementara dan tidak ada inventory. Masing-masing produk yang keluar dari gudang harus sama dengan masing-masing produk yang masuk ke gudang. Pada batasan ini dimungkinkan adanya pengiriman produk antar gudang.

<img src="https://user-images.githubusercontent.com/61647791/147673078-3b0ce19c-223b-4204-83d9-3490de8e1813.png" />

In [11]:
#Warehouse Cross Docking Constrain
for c in product:
    for w in warehouse:
        model += lpSum(x[c,w,j] for j in warehouse+retailer if w!=j) == lpSum(x[c,i,w] for i in plant+warehouse if i!=w)

#### Constrain 3: Retailer Demand
Selisih masing-masing produk yang masuk dan keluar retailer harus lebih dari sama dengan demand-nya. Jika kurang dari sama dengan dikhawatirkan demand tidak dapat terpenuhi. Pada batasan ini dimungkinkan adanya pengiriman produk antar retailer.

<img src="https://user-images.githubusercontent.com/61647791/147673635-a56f2caa-5dec-425c-b329-21a0275fc1e7.png" />

In [12]:
#Retailer Demand
for c in product:
    for r in retailer:
        model += lpSum(x[c,i,r] for i in plant+warehouse+retailer if i!=r) - lpSum(x[c,r,j] for j in retailer if r!=j) >= demand.loc[r][c]

#### Constrain 4: Transportation Capacity
Ingat bahwa kedua produk memiliki ukuran yang berbeda dan saling berkompetisi dalam memanfaatkan kapasitas transportasi. 

<img src="https://user-images.githubusercontent.com/61647791/147674325-e241f44e-b893-4f17-bcd7-49bd8eb9734f.png" />


In [13]:
#Transportation Capacity Constraint
ij_keys = [(i,j) for i in cost.index for j in cost.index if i!=j and (i,j) not in nan_keys]

for i,j in ij_keys:
    model += lpSum(x[c,i,j]*weight[c] for c in product) <=300

## 5. Menyelesaikan Model dan Melihat Hasil Optimasi

In [14]:
#Jika hasilnya 1 artinya model sudah optimal
model.solve()

1

In [15]:
#Total biaya yang dibutuhkan
value(model.objective)

4890.0

In [16]:
#Data pengiriman produk
origin_A = []
destination_A = []
flow_A = []

origin_B = []
destination_B = []
flow_B = []

for c,i,j in keys:
    if x[c,i,j].varValue >0:
        if c == "Product A":
            origin_A.append(i)
            destination_A.append(j)
            flow_A.append(x[c,i,j].varValue)
        else:
            origin_B.append(i)
            destination_B.append(j)
            flow_B.append(x[c,i,j].varValue)
        
product_A = pd.DataFrame({"Origin":origin_A,
                          "Destination":destination_A,
                          "Flow A":flow_A})

product_B = pd.DataFrame({"Origin":origin_B,
                          "Destination":destination_B,
                          "Flow B":flow_B})

result = pd.merge(product_A, product_B, how="outer", left_on=["Origin","Destination"], right_on=["Origin","Destination"]).fillna(0)
result["Flow A"] = result["Flow A"].astype("int")
result["Total Flow"] = result["Flow A"] + 2* result["Flow B"]
result

,Origin,Destination,Flow A,Flow B,Total Flow
0,Plant 1,Plant 3,90,90,270
1,Plant 2,Retailer 1,150,60,270
2,Plant 3,Warehouse 2,140,80,300
3,Warehouse 2,Retailer 1,140,80,300
4,Retailer 1,Retailer 2,90,90,270
5,Plant 2,Warehouse 1,0,90,180
6,Plant 3,Warehouse 1,0,60,120
7,Warehouse 1,Retailer 1,0,150,300


## Kesimpulan:
Berdasarkan hasil optimasi di atas total biaya pengiriman yang dikeluarkan sebesar $4890 dengan konfigurasi jaringan rantai pasok sebagai berikut:




<img src="https://user-images.githubusercontent.com/61647791/147676517-1a7d268d-1406-44f0-8c09-264f306fcbd1.png" />